# Turbulent box tutorial

## Instalation

The tools we use are RAMSES and Osyris. More information on the codes here:
- https://github.com/nvaytet/osyris
- https://github.com/ramses-organisation/ramses

To install osyris, simply do

In [ ]:
%pip install osyris==2.11

You can also install it by cloning directly the source. In this case, please follow the instruction on the GitHub webpage (https://osyris.readthedocs.io/en/latest/installation.html).
Note that for now the tutorials are written for Osyris 2.11 and don't work with Osyris 3.

To download RAMSES, do in your terminal

In [ ]:
%%bash
git clone https://github.com/ramses-organisation/ramses.git

To compile ramses, you need at least a fortran compiler, either fortran (GNU compiler) or ifort (Intel compiler). You can get more information on the GNU compiler here:
https://gcc.gnu.org/fortran/
https://fortran-lang.org/learn/os_setup/install_gfortran

To run the turbulence driving simulations, you also need the FFTW library. Please follow what is explained in the file ramses/turb/README.

## Run RAMSES

Once you downloaded RAMSES, go in the main directory (`cd ramses`)

### You can first run the test suite:

For each test, there is first a compilation (using the config.txt file of each test), then the code runs (using the namelist files .nml) and finally results are    plotted using the plot-test-name.py routines.

There are various options for the test suite:    
```bash
#      - Run the suite in parallel (on 4 cpus):
              ./run_test_suite.sh -p 4
#      - Do not delete results data:
             ./run_test_suite.sh -d
#      - Run in verbose mode:
              ./run_test_suite.sh -v
#      - Select test number (for tests 3 to 5, and 10):
              ./run_test_suite.sh -t 3-5,10
#      - Run all tests in mhd directory:
              ./run_test_suite.sh -t mhd
#      - Run quick test suite:
              ./run_test_suite.sh -q
```
The option `-d` is quite useful if you want to work more on the data than what is proposed in the test suite. I would also recommend to use the `-y` option to use python for visualisation.

The turbulence driving test is the test number [16]. You will find in the directory `tests/turb/driving`  the routine plot-driving.py that can be an alternative to the use of osyris. In particular, you can produce column density maps that could be applied to the tools you learned last week!

Of course, you can also play with all the other tests! Note that you can find information on the namelist parameters you can play with here: https://ramses-organisation.readthedocs.io/en/latest/wiki/Runtime_Parameters.html

In [ ]:
%%bash
cd ramses/tests
./run_test_suite.sh -t turb -p 4 

This will run the test suite for the turbulence test case with 4 MPI processes.

### You can then compile directly ramses

Go to the bin directory and update the Makefile: NDIM=3 & USE_TURB=1 

_(note that this corresponds to the FLAGS indicated in the file config.txt of the turbulence driving test)._

**Then**, still in the bin directory


In [ ]:
%%bash
cd ramses/bin
make clean
make NDIM=3 USE_TURB=1 MPI=1 

You should have a file ramses3d that has been created (if not, double check that you updated the Makefile correctly).

In [ ]:
%%bash
ls ramses/bin

You can then run ramses. 
If you have MPI installed you can run on 4 processors: `mpirun -np 4 ramses3d driving.nml`

In [ ]:
%%bash 
cp ramses/bin/ramses3d ramses/tests/turb/driving/
cd ramses/tests/turb/driving/
./ramses3d driving.nml 

You can now play with the namelist parameters. You can change the resolution (`levelmin=5`, `levelmax=5` for a 32^3 resolution simulation).
You can add more outputs using the parameter foutput in &OUTPUT_PARAMS


```config
&OUTPUT_PARAMS
noutput=2
tout=0.0,0.5
foutput=25 ! Output every 25 steps
/
```



Enjoy!

## Use Osyris to visualise the data

You can check the documentation of osyris [here](https://osyris.readthedocs.io/en/stable/index.html).

### Plot a density slice


Modify the code below to 
- Add an extra layer with the velocity as arrows
- Change the spatial units to pc (you can use the parameter `dx`)
- Change the density unit to solar masses to pc^2
- Change the colormap to inferno

The final output should look like that:

![expected_slice](slice.png)

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

data = osyris.RamsesDataset(2, path="ramses/tests/turb/driving/").load() # load output number 20. 
center = osyris.Vector(x=2, y=2, z=2, unit="pc") # Choose the center of the box

osyris.map(
    data["mesh"].layer("density"),  # layer 1 : density
    norm="log", origin=center, direction="z")